## Text Retrieval Augmented Generation (RAG) with LangChain

In this notebook we will cover how to use LangChain retrievers to do Text RAG with LangChain. In the documents folder we have a number of scientific papers that we will store in an in memory DB and use in our RAG chain.

Let's get started.

### Setup

In [ ]:
%pip install langchain_community faiss_cpu pypdf "langchain_aws>=0.2.9"

### Process our PDFs

We will use the PyPDFDirectorLoader to load all of the documents in the folder. Then, take advantage of the RecursiveCharacterTextSplitter to chunk the documents. This process decomposes large files into chunks that can be fed through our embedding model

In [ ]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

def _pdf_to_chunks(doc_path: str):
    loader = PyPDFDirectoryLoader(doc_path)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    return text_splitter.split_documents(documents)

### Create the Vector Store

Now that we have the document chunks we can embed them using Bedrock Embeddings (the default embeddings model is Titan Text Embeddings) and store them in a FAISS in memory vectory store

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_aws import BedrockEmbeddings

text_chunks = _pdf_to_chunks("./media/")
vectorstore = FAISS.from_documents(documents=text_chunks, embedding=BedrockEmbeddings())
retriever = vectorstore.as_retriever()

### Create the RAG Prompt

We will now create a prompt that will take the user query and the retrieved context to be sent to the model for the final generation. The provided model instructions show best practices to minimize hallucinations and to have citations. 

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
"""
In this session, the model has access to search results and a user's question, your job is to answer the user's question using only information from the search results.

Model Instructions:
- You should provide concise answer to simple questions when the answer is directly contained in search results, but when comes to yes/no question, provide some details.
- In case the question requires multi-hop reasoning, you should find relevant information from search results and summarize the answer based on relevant information with logical reasoning.
- If the search results do not contain information that can answer the question, please state that you could not find an exact answer to the question, and if search results are completely irrelevant, say that you could not find an exact answer, then summarize search results.
- Remember to add a citation to the end of your response using markers like [1], [2], [3], etc for the corresponding passage supports the response. 
- The citations should be able to be rendered in markdown and each citation should be rendered on a new line
- DO NOT USE INFORMATION THAT IS NOT IN SEARCH RESULTS!

{question} 
Resource: {context}
"""
)

### Context Formatting

We create a helper function that will take the context from the retriever and format it in a way that provides both the source information and content for the model to reference when doing the final generation

In [ ]:
def _format_docs(docs):
    return [{"source_metadata": i.metadata, "source_content": i.page_content} for i in docs]

### Create the RAG Chain

Finally we construct our RAG chain and can invoke the full retriever workflow. 

In [ ]:
from langchain_aws import ChatBedrockConverse

from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

llm = ChatBedrockConverse(
    model_id="us.amazon.nova-lite-v1:0",
    temperature=0.7,
    max_tokens=2000
)

rag_chain = (
    {"context": retriever | _format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print(
    rag_chain.invoke(
        "What are the benefits of using multi-agent workflows for translating literary texts. Site your sources"
    )
)